<a href="https://colab.research.google.com/github/harvey4878/Harvey/blob/main/%5BUpstage_AI_LAB_01%5D_Spaceship_Titanic_ML_PJT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning 프로젝트 수행을 위한 코드 구조화

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [2]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.1 MB/s eta 0:00:00


In [3]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [5]:
# 데이터를 불러옵니다.
train = pd.read_csv('/content/drive/MyDrive/spaceship-titanic/train.csv')
test = pd.read_csv('/content/drive/MyDrive/spaceship-titanic/test.csv')

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [6]:
## 1. 결측치 체크  --> 특정 column이 많은 결측치를 포함하고 있는지!
train[train.isnull().any(axis=1)] # boolean mask ## 결측치를 포함하고 있는 데이터가 2087개.
## 2. dtype이 object인 column들 체크 (str)
cat_cols = train.columns[train.dtypes == 'object']    ## mode로
num_cols = train.columns[~(train.dtypes == 'object')] ## mean으로
## 3. target value 정보를 체크 -----> class imbalance 체크! (이진분류) 10:1 이상인 경우
train.Transported.value_counts()

True     4378
False    4315
Name: Transported, dtype: int64

In [7]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [8]:
pd.pivot_table(data=train, index=['HomePlanet', 'VIP'], values='Transported', aggfunc='mean')
# passengerId : group_num(4자리, usually family but not always) + id(그룹내에서 2자리)
# HomePlanet, Desitination : categorical feature.
# Cabin (side deck / otherwise)
# CryoSleep, VIP : bool
# Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck : 쓴 돈.
# Name : 이름
# Transported(y) : True / False

Transported
HomePlanet VIP               
Earth      False     0.424337
Europa     False     0.670072
           True      0.488550
Mars       False     0.534180
           True      0.158730

이런 식으로 여러가지 그래프를 그려가며, 데이터에 대한 인사이트를 얻습니다!

### 3. 전처리

#### 결측치 처리

In [9]:
## TO-DO ##
## group 정보를 passenger_Id로부터 추출해서, group_size가 4 이상인 고객들은 in_large_group 이라는 column에 1,
##                                     나머지는 0으로 하는 feature를 만들어보세요!
group = train.PassengerId.apply(lambda x:x[:4])
temp = group.value_counts()
large_group_num = temp[temp >= 4].index
train['in_large_group'] = group.isin(large_group_num) * 1
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,in_large_group
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,0
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0


In [10]:
pd.pivot_table(data=train, index='in_large_group', values='Transported', aggfunc=['count', 'sum', 'mean'])

,count,sum,mean
,Transported,Transported,Transported
in_large_group,,,
0,7507,3684,0.490742
1,1186,694,0.585160


In [11]:
# 결측치가 있는 column
# dtype이 object인 데이터들을 수치화(encoding)
# 결측치 제거 방법
# 1)
# 2)
# 3)

## TO-DO ##
## cat_cols에 대해서는 mode를 계산해서 채워주고, num_cols에 대해서는 mean을 계산해서 채워줌.
modes = {}
means = {}
for col in cat_cols:
    modes[col] = train[col].mode()[0]
    train[col] = train[col].fillna(modes[col])

for col in num_cols:
    means[col] = train[col].mean()
    train[col] = train[col].fillna(means[col])

# categorical feature encoding
train = pd.get_dummies(data=train, columns=['HomePlanet', 'Destination'])
drop_cols = ['PassengerId', 'Cabin', 'Name']
train = train.drop(columns=drop_cols)
train # 8693 x 16

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,in_large_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,0,1,0,0,0,1
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0,1,0,0,0,0,1
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,0,1,0,0,0,1
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,0,1,0,0,0,1
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,0,0,1,0,1,0,0
8689,True,18.0,False,0.0,0.0,0.0,0.0,0.0,False,0,1,0,0,0,1,0
8690,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,0,1,0,0,0,0,1
8691,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,0,0,1,0,1,0,0


### 4. 학습 데이터 분할

In [12]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns='Transported') # feature vector
y = train.Transported                 # target value

# 트테트테
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(6954, 15) (6954,) (1739, 15) (1739,)


### 5. 학습 및 평가

In [13]:
# model = RandomForestClassifier(n_estimators=200,
#                                max_depth=7,
#                                min_samples_split=10,
#                                min_samples_leaf=5,
#                                max_features=0.7,
#                                random_state=42,
#                                n_jobs=-1)
from xgboost import XGBClassifier
model = XGBClassifier()

In [14]:
print("\nFitting RandomForest...")
model.fit(X_train, y_train)


Fitting RandomForest...


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [15]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = accuracy_score

In [16]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.8865
Validation Score : 0.7849


### 6. Hyper-parameter Tuning

In [17]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators' : [50, 100],
    'max_depth' : [6, 7, 8],
    'min_samples_split' : [2, 10],
    'min_samples_leaf' : [1, 5],
    'max_features' : [0.5, 0.65],
    'random_state' : [42]
} # 3 x 2 x 2 x 2 x 2 = 48개.

gcv = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=5, verbose=1)

gcv.fit(X_train, y_train)
print("Best Estimator : ", gcv.best_estimator_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Estimator :  RandomForestClassifier(max_depth=8, max_features=0.5, min_samples_split=10,
                       random_state=42)


In [18]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_train)
gcv_pred_val = gcv.predict(X_val)

gcv_train_score = evaluation_metric(y_train, gcv_pred_train)
gcv_val_score = evaluation_metric(y_val, gcv_pred_val)

print("Train ACC Score : %.4f" % gcv_train_score)
print("Validation ACC Score : %.4f" % gcv_val_score)

Prediction with Best Estimator
Train ACC Score : 0.8172
Validation ACC Score : 0.7815


> optuna를 사용해봅시다 !

In [26]:
def optimizer(trial):
    # 조절할 hyper-parameter 조합을 적어줍니다.  # bayesian optimization
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    #max_depth = trial.suggest_int('max_depth', 5, 10)
    min_samples_split = trial.suggest_categorical('min_samples_split', [2, 10, 25])
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_float('max_features', 0.5, 0.8)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=10,
                                   min_samples_split=min_samples_split,
                                   min_samples_leaf=min_samples_leaf,
                                   max_features=max_features,
                                   random_state=42)

    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = evaluation_metric(y_val, preds)

    return score

In [27]:
study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(optimizer, n_trials=10)

[I 2023-11-28 21:50:55,309] A new study created in memory with name: no-name-1a601005-065e-45f4-95fb-41317a8fbf8d
[I 2023-11-28 21:50:57,742] Trial 0 finished with value: 0.78205865439908 and parameters: {'n_estimators': 124, 'min_samples_split': 25, 'min_samples_leaf': 4, 'max_features': 0.6842922752863144}. Best is trial 0 with value: 0.78205865439908.
[I 2023-11-28 21:50:58,780] Trial 1 finished with value: 0.7837837837837838 and parameters: {'n_estimators': 54, 'min_samples_split': 25, 'min_samples_leaf': 9, 'max_features': 0.6235202056259576}. Best is trial 1 with value: 0.7837837837837838.
[I 2023-11-28 21:51:04,072] Trial 2 finished with value: 0.7866589994249569 and parameters: {'n_estimators': 181, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 0.6972287376612355}. Best is trial 2 with value: 0.7866589994249569.
[I 2023-11-28 21:51:10,450] Trial 3 finished with value: 0.7872340425531915 and parameters: {'n_estimators': 88, 'min_samples_split': 2, 'min_samples_l

In [28]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_max_features,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.782059,2023-11-28 21:50:55.315897,2023-11-28 21:50:57.741539,0 days 00:00:02.425642,0.684292,4,25,124,COMPLETE
1,1,0.783784,2023-11-28 21:50:57.743341,2023-11-28 21:50:58.779891,0 days 00:00:01.036550,0.623520,9,25,54,COMPLETE
2,2,0.786659,2023-11-28 21:50:58.788344,2023-11-28 21:51:04.072453,0 days 00:00:05.284109,0.697229,3,2,181,COMPLETE
3,3,0.787234,2023-11-28 21:51:04.088707,2023-11-28 21:51:10.450065,0 days 00:00:06.361358,0.782658,8,2,88,COMPLETE
4,4,0.784934,2023-11-28 21:51:10.455085,2023-11-28 21:51:13.467072,0 days 00:00:03.011987,0.542596,5,10,118,COMPLETE
5,5,0.780334,2023-11-28 21:51:13.473245,2023-11-28 21:51:16.455216,0 days 00:00:02.981971,0.795702,4,25,124,COMPLETE
6,6,0.782059,2023-11-28 21:51:16.463429,2023-11-28 21:51:17.981802,0 days 00:00:01.518373,0.539339,6,2,122,COMPLETE
7,7,0.782059,2023-11-28 21:51:17.984240,2023-11-28 21:51:20.260123,0 days 00:00:02.275883,0.776725,4,25,196,COMPLETE
8,8,0.781484,2023-11-28 21:51:20.262367,2023-11-28 21:51:23.000577,0 days 00:00:02.738210,0.618677,6,25,194,COMPLETE
9,9,0.783209,2023-11-28 21:51:23.006110,2023-11-28 21:51:26.151591,0 days 00:00:03.145481,0.793503,10,2,134,COMPLETE


In [29]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.7872
Best params:  {'n_estimators': 88, 'min_samples_split': 2, 'min_samples_leaf': 8, 'max_features': 0.7826577179847334}


In [30]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [31]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

In [32]:
# model finalization
print("Validation ACC")
best_params = study.best_params
best_model = RandomForestClassifier(**best_params,max_depth=10,
                                    random_state=42)
best_model.fit(X_train, y_train)
print("Validation Score : %.3f" % evaluation_metric(y_val, best_model.predict(X_val)))

Validation ACC
Validation Score : 0.787


### 7. 테스트 및 제출 파일 생성

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [41]:
## X_test 만들기 -> traindata에 사용한 전처리 기법을 그대로 사용하기!
# TODO : X_train와 column 개수도 같고, column의 순서도 같은 X_test를 만드세요!
X_test = test
group = X_test.PassengerId.apply(lambda x:x[:4])
temp= group.value_counts()
large_group_num = temp[temp>=4].index
X_test['in_large_group']= group.isin(large_group_num)*1

In [47]:
X_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,in_large_group
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,0
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,0
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,0
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,0


In [48]:
for col in cat_cols:
  X_test[col]=X_test[col].fillna(modes[col])
for col in num_cols:
  if col == 'Transported':
    continue
  X_test[col] =X_test[col].fillna(means[col])


In [51]:
X_test

,PassengerId,CryoSleep,Cabin,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,in_large_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,G/3/S,27.00000,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0,1,0,0,0,0,1
1,0018_01,False,F/4/S,19.00000,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0,1,0,0,0,0,1
2,0019_01,True,C/0/S,31.00000,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0,0,1,0,1,0,0
3,0021_01,False,C/1/S,38.00000,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0,0,1,0,0,0,1
4,0023_01,False,F/5/S,20.00000,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,G/1496/S,34.00000,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,0,1,0,0,0,0,1
4273,9269_01,False,G/734/S,42.00000,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,0,1,0,0,0,0,1
4274,9271_01,True,D/296/P,28.82793,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,0,0,0,1,1,0,0
4275,9273_01,False,D/297/P,28.82793,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,0,0,1,0,0,0,1


In [52]:
##Categorical features
drop_cols=['PassengerId','Cabin','Name']
X_test = X_test.drop(columns=drop_cols)
X_test

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,in_large_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,27.00000,False,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1
1,False,19.00000,False,0.0,9.0,0.0,2823.0,0.0,0,1,0,0,0,0,1
2,True,31.00000,False,0.0,0.0,0.0,0.0,0.0,0,0,1,0,1,0,0
3,False,38.00000,False,0.0,6652.0,0.0,181.0,585.0,0,0,1,0,0,0,1
4,False,20.00000,False,10.0,0.0,635.0,0.0,0.0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,True,34.00000,False,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,1
4273,False,42.00000,False,0.0,847.0,17.0,10.0,144.0,0,1,0,0,0,0,1
4274,True,28.82793,False,0.0,0.0,0.0,0.0,0.0,0,0,0,1,1,0,0
4275,False,28.82793,False,0.0,2680.0,0.0,0.0,523.0,0,0,1,0,0,0,1


In [53]:
# warning도 error도 안나와야 제대로된 예측값이 나옴.
preds = best_model.predict(X_test)
preds

array([ True, False,  True, ...,  True,  True,  True])

In [55]:
submission = pd.read_csv('/content/drive/MyDrive/spaceship-titanic/sample_submission.csv')
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [56]:
submission['Transported'] = preds
submission.to_csv("submission.csv", index=False)